In [ ]:
pip install -U langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.2/413.2 kB 11.1 MB/s eta 0:00:0000:01
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.12 requires async-timeout<5.0.0,>=4.0.0; python_version < "3.11", but you have async-timeout 5.0.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import random
import json
from datasets import load_dataset, Dataset
from langchain_google_genai import ChatGoogleGenerativeAI
import os
import time

**API KEY LINK:** [HERE](https://aistudio.google.com/app/apikey)

In [ ]:
# Load API Key (ensure it's set in your Kaggle environment)
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
gemini_secret = user_secrets.get_secret("GEMINI_API_KEY")

In [ ]:
# Initialize the LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.4,
    max_tokens=None,
    api_key=gemini_secret,
)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# Load dataset from Hugging Face
dataset = load_dataset("EmoCareAI/Psych8k")
train_data = dataset["train"]

README.md:   0%|          | 0.00/564 [00:00<?, ?B/s]

Alexander_Street_shareGPT_2.0.json:   0%|          | 0.00/6.58M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8187 [00:00<?, ? examples/s]

In [ ]:
# Randomly sample 100 data points
# sample_data = random.sample(list(train_data), 5)

In [ ]:
# ------------------------ORIGINAL
# # Define the transformation function
# def transform_dialogue(input_text, output_text):
#     prompt = f"""
#     I want you to act as both a psychiatrist and a dialogue rewriter. Your task is to transform the given single-turn dialogue into a multi-turn conversation that feels natural, interactive, and realistic.

#     **Requirements:**
#     1. The patient should NOT repeat their entire original statement in the first turn. Instead, their responses should be broken into smaller parts throughout the conversation.
#     2. The psychiatrist should ask relevant follow-up questions that encourage the patient to gradually share their experience.
#     3. The conversation should have at least 10 exchanges (5 turns each for the patient and psychiatrist).
#     4. Do NOT introduce new details that are not explicitly mentioned in the single-turn dialogue. If information is missing (e.g., duration, triggers), the psychiatrist should ask the patient rather than assume.
#     5. The psychiatrist should use empathetic and professional language while maintaining a supportive tone.
#     6. Each turn should be clearly marked with **"Patient:"** or **"Psychiatrist:"**.

#     **Single-Turn Dialogue:**
#     Patient: {input_text}
#     Psychiatrist: {output_text}

#     Now, please generate the expanded multi-turn dialogue.
#     """

#     response = llm.invoke(prompt)
#     return response.content

In [ ]:
# --------------------------FEW_SHOT
# Define the transformation function
def transform_dialogue(input_text, output_text):
    prompt = f"""
    I want you to act as both a psychiatrist and a dialogue rewriter. Your task is to transform the given single-turn dialogue into a multi-turn conversation that feels natural, interactive, and realistic.

    **Requirements:**
    1. The patient should NOT repeat their entire original statement in the first turn. Instead, their responses should be broken into smaller parts throughout the conversation.
    2. The psychiatrist should ask relevant follow-up questions that encourage the patient to gradually share their experience.
    3. The conversation should have at least 10 exchanges (5 turns each for the patient and psychiatrist).
    4. Do NOT introduce new details that are not explicitly mentioned in the single-turn dialogue. If information is missing (e.g., duration, triggers), the psychiatrist should ask the patient rather than assume.
    5. The conversation should only contain standard English text and avoid any special characters (e.g., symbols, emojis, or non-English characters) within the dialogue.
    6. The psychiatrist should use empathetic and professional language while maintaining a supportive tone.
    7. Each turn should be clearly marked with **"Patient:"** or **"Psychiatrist:"**.


    **Example 1:**

    **Single-Turn Dialogue:**
    Patient: "Lately, I've been feeling really disconnected from reality. I often find it hard to distinguish between what's real and what's not. For example, last week, I was having dinner with my family, and I suddenly thought my brother was about to attack me. I know it doesn't make any sense, but it felt real in that moment. This has happened a few times, but not all the time. I'm also having a hard time concentrating and remembering things that have happened recently. I've had some depressive episodes in the past, but this seems a bit different to me. Can you help me figure out what's going on?"

    Psychiatrist: "Thank you for sharing your experience with me. It sounds like you've been dealing with a mix of symptoms, and I can understand how that can be concerning. Before I can provide any feedback or guidance, can you please tell me more about your experience? For example, how long have these feelings of disconnection been happening, and are there any other symptoms you've noticed recently? It would also be helpful to know about any stressors in your life right now or any history of mental health issues in your family. This information will help me understand your situation better."

    **Multi-turn Dialogue:**
    Patient: Lately, I’ve been feeling really disconnected from reality. It’s like I can’t always tell what’s real and what’s not.
    Psychiatrist: That sounds unsettling. Can you tell me more? When did you first start noticing these feelings?
    Patient: I’m not exactly sure, but I know it’s been happening more frequently.
    Psychiatrist: I see. You mentioned difficulty distinguishing reality. Could you describe an example of when this happened?
    Patient: Last week, I was having dinner with my family, and for a moment, I thought my brother was about to attack me. It didn’t make any sense, but it felt real at the time.
    Psychiatrist: That must have been a frightening experience. How often do these kinds of episodes occur?
    Patient: It’s not constant, but it’s happened a few times. Some days, I feel completely fine.
    Psychiatrist: I understand. Apart from these experiences, have you noticed any other changes—like in your concentration or memory?
    Patient: Yes, I find it hard to focus and sometimes forget things that just happened. It’s frustrating because I used to be really sharp.
    Psychiatrist: That must be difficult. Have you also noticed changes in your mood or energy levels?
    Patient: Well, I’ve had depressive episodes in the past, but this feels different. I don’t feel just sad—I feel like I’m not fully present in my own life.
    Psychiatrist: I appreciate you sharing that. This sense of disconnection, along with memory issues, could have multiple causes. Have you been under any unusual stress recently?
    Patient: A bit, but nothing extreme. I don’t know if that’s what’s causing this.
    Psychiatrist: That’s helpful to know. We’ll explore this further to understand what might be going on and how to help you feel more grounded.

    **Now, based on the following single-turn dialogue, generate a multi-turn conversation following the same structure.**

    **Single-Turn Dialogue:**
    Patient: {input_text}
    Psychiatrist: {output_text}

    **Multi-turn Dialogue:**
    """

    response = llm.invoke(prompt)
    return response.content


In [ ]:
# -------------CHECKING
# Process the 5-sample data
# output_data = []
# for item in sample_data:
#     input_text = item["input"]
#     output_text = item["output"]
#     expanded_dialogue = transform_dialogue(input_text, output_text)
#     output_data.append({"input": input_text, "output": output_text, "expanded_dialogue": expanded_dialogue})


In [ ]:
# Save to CSV
# output_df = pd.DataFrame(output_data)
# # output_df.to_csv("expanded_psych8k_sample.csv", index=False)
# print("Saved 20-sample data to CSV")

In [ ]:
def process_dataset_range(start_idx, end_idx):
    subset = {
        "input": train_data["input"][start_idx:end_idx],
        "output": train_data["output"][start_idx:end_idx],
        "instruction": train_data["instruction"][start_idx:end_idx]
    }

    processed_data = []
    for i in range(len(subset["input"])):
        input_text = subset["input"][i]
        output_text = subset["output"][i]
        time.sleep(3)
        expanded_dialogue = transform_dialogue(input_text, output_text)  # Assuming this function exists

        processed_data.append({
            "input": input_text,
            "output": output_text,
            "expanded_dialogue": expanded_dialogue
        })
        print(f"Interation {i} done.")

    # Save as CSV
    csv_filename = f"expanded_psych8k_{start_idx}_{end_idx}.csv"
    df=pd.DataFrame(processed_data)
    df.to_csv(csv_filename, index=False)
    print(f"Saved dataset to {csv_filename}")

    # # Save in Hugging Face format (NEED TO CHANGE)
    # dataset = Dataset.from_list(processed_data)
    # hf_dir = f"expanded_psych8k_{start_idx}_{end_idx}"
    # dataset.save_to_disk(hf_dir)
    # print(f"Saved dataset in Hugging Face format at {hf_dir}")
    return df

In [ ]:
# Example Usage: Process dataset from index 1000 to 2000
output_df=process_dataset_range(4901, 520)

Interation 0 done.
Interation 1 done.
Interation 2 done.
Interation 3 done.
Interation 4 done.
Interation 5 done.
Interation 6 done.
Interation 7 done.
Interation 8 done.
Interation 9 done.
Interation 10 done.
Interation 11 done.
Interation 12 done.
Interation 13 done.
Interation 14 done.
Interation 15 done.
Interation 16 done.
Interation 17 done.
Interation 18 done.
Interation 19 done.
Interation 20 done.
Interation 21 done.
Interation 22 done.
Interation 23 done.
Interation 24 done.
Interation 25 done.
Interation 26 done.
Interation 27 done.
Interation 28 done.
Interation 29 done.
Interation 30 done.
Interation 31 done.
Interation 32 done.
Interation 33 done.
Interation 34 done.
Interation 35 done.
Interation 36 done.
Interation 37 done.
Interation 38 done.
Interation 39 done.
Interation 40 done.
Interation 41 done.
Interation 42 done.
Interation 43 done.
Interation 44 done.
Interation 45 done.
Interation 46 done.
Interation 47 done.
Interation 48 done.
Interation 49 done.
Interation

ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).

In [ ]:
Example Usage: Process dataset from index 1000 to 2000
output_df=process_dataset_range(5201, 5500)

In [ ]:
# Example Usage: Process dataset from index 1000 to 2000
output_df=process_dataset_range(5501, 5750)

Interation 0 done.
Interation 1 done.
Interation 2 done.
Interation 3 done.
Interation 4 done.
Interation 5 done.
Interation 6 done.
Interation 7 done.
Interation 8 done.
Interation 9 done.
Interation 10 done.
Interation 11 done.
Interation 12 done.
Interation 13 done.
Interation 14 done.
Interation 15 done.
Interation 16 done.
Interation 17 done.
Interation 18 done.
Interation 19 done.
Interation 20 done.
Interation 21 done.
Interation 22 done.
Interation 23 done.
Interation 24 done.
Interation 25 done.
Interation 26 done.
Interation 27 done.
Interation 28 done.
Interation 29 done.
Interation 30 done.
Interation 31 done.
Interation 32 done.
Interation 33 done.
Interation 34 done.
Interation 35 done.
Interation 36 done.
Interation 37 done.
Interation 38 done.
Interation 39 done.
Interation 40 done.
Interation 41 done.
Interation 42 done.
Interation 43 done.
Interation 44 done.
Interation 45 done.
Interation 46 done.
Interation 47 done.
Interation 48 done.
Interation 49 done.
Interation

In [ ]:
output_df=process_dataset_range(5751, 6000)

# Printing

In [ ]:
# Randomly select 2 rows
# random_rows = output_df.sample(n=5, random_state=None)  # Remove random_state for true randomness

# # Print each row with formatted expanded_dialogue
# for _, row in random_rows.iterrows():
#     print(f"Input: {row['input']}\n")
#     print(f"Output: {row['output']}\n")
#     print(f"Expanded Dialogue:\n{row['expanded_dialogue']}\n")
#     print("=" * 80)  # Separator for readability